#Description
This notebook aims to get the NPL information from BACEN timeseries system

# Setup

In [3]:
import pandas as pd
import requests
import plotly.express as px
import io
import datetime

# Data

In [ ]:
requests_dict = {
    'npl_individuals_total' : '21033',
    'npl_overdraft' : '21034',
    'npl_personal_loans' : '21035',
    'npl_private_payroll' : '21037',
    'npl_public_sector_payroll' : '21038',
    'npl_retirees_payroll' : '21039',
    'npl_payroll_total' : '21041',
    'npl_credit_cards' : '21050',
    
}

def getBacenSeries(dict_series: dict, end_date: str, beg_date: str = '01/01/1995'):
    result = None
    for name_i, code_i in dict_series.items():
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{code_i}/dados?formato=csv&dataInicial={beg_date}&dataFinal={end_date}"
        df = pd.read_csv(url, sep=";", decimal=",", date_format="%d/%m/%Y", parse_dates=[0])
        df = df.rename({'valor':name_i, 'data' : 'date'}, axis='columns')
        if result is None:
            result = df
        else:
            result = result.merge(df, on='date', how='left')
    return result
        
npls_df = getBacenSeries(requests_dict, "31/12/2025")
npls_df.head()

,date,npl_individuals_total,npl_overdraft,npl_personal_loans,npl_private_payroll,npl_public_sector_payroll,npl_retirees_payroll,npl_payroll_total,npl_credit_cards
0,2011-03-01,6.83,5.49,5.67,5.75,5.21,1.79,4.70,4.17
1,2011-04-01,6.78,5.67,5.39,6.38,4.71,1.75,4.43,4.16
2,2011-05-01,6.62,5.19,5.34,4.86,4.55,1.83,4.28,4.19
3,2011-06-01,6.61,5.76,5.28,4.81,4.43,1.74,4.20,4.28
4,2011-07-01,6.53,6.41,5.02,4.75,4.06,1.85,3.97,4.18


In [3]:
data_chart = npls_df.melt(id_vars='date')
fig = px.line(data_chart,
              x='date',
              y='value',
              color='variable',
              template='none',
              width=1000)
fig

In [27]:
requests_dict = {
    "ibc_br" : "24363",
    'quarterly_gdp' : '22099',
}

def getBacenSeries(dict_series: dict, end_date: str, beg_date: str = '01/01/1995'):
    result = None
    for name_i, code_i in dict_series.items():
        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{code_i}/dados?formato=csv&dataInicial={beg_date}&dataFinal={end_date}"
        df = pd.read_csv(url, sep=";", decimal=",", date_format="%d/%m/%Y", parse_dates=[0])
        df = df.rename({'valor':name_i, 'data' : 'date'}, axis='columns')
        if result is None:
            result = df
        else:
            result = result.merge(df, on='date', how='left')
    return result

def withYearlyVariation(df: pd.DataFrame, col_name: str):
    df_temp = df.copy()
    df_temp['previous_year'] = df_temp['date'] - pd.offsets.DateOffset(years=1)
    df_temp = df_temp.merge(df_temp[['date', col_name]].rename({col_name:f'{col_name}_previous_year'}, axis="columns"),
                            right_on="date",
                            left_on="previous_year",
                            how="left")
    df_temp[f'{col_name}_annual_growth'] = round((df_temp[col_name] / df_temp[f'{col_name}_previous_year'] - 1)*100,2)
    result = df.copy()
    result[f'{col_name}_annual_growth'] = df_temp[f'{col_name}_annual_growth']
    return result

gdp_df = getBacenSeries(requests_dict, "31/12/2025")\
            .pipe(withYearlyVariation, "quarterly_gdp")\
            .pipe(withYearlyVariation, "ibc_br")

gdp_df.tail()

,date,ibc_br,quarterly_gdp,quarterly_gdp_annual_growth,ibc_br_annual_growth
265,2025-02-01,106.80038,NaN,NaN,4.42
266,2025-03-01,113.80032,NaN,NaN,3.66
267,2025-04-01,112.23595,194.65,2.22,2.69
268,2025-05-01,108.40943,NaN,NaN,3.53
269,2025-06-01,107.26420,NaN,NaN,1.38


In [26]:
data_chart = gdp_df.melt(id_vars="date", value_vars=["quarterly_gdp_annual_growth", "ibc_br_annual_growth"])
fig = px.line(data_chart,
              x="date",
              y="value",
              color="variable",
              template="none",
              width=1000)
fig

In [ ]:


gdp_df.pipe(withYearlyVariation, "quarterly_gdp")

,date,quarterly_gdp,quarterly_gdp_annual_growth
0,1995-04-01,100.00,NaN
1,1995-07-01,100.00,NaN
2,1995-10-01,100.00,NaN
3,1996-01-01,96.84,NaN
4,1996-04-01,100.12,0.12
...,...,...,...
116,2024-04-01,190.43,3.34
117,2024-07-01,194.94,4.05
118,2024-10-01,189.51,3.61
119,2025-01-01,190.65,2.85


Series available in the World Economic Outlook [WEO](https://data.imf.org/en/Datasets#t=coveo117bcfc4&sort=%40idata_publication_date%20descending&numberOfResults=50&f:Topics=[Gross%20domestic%20product%20(GDP)])

| SERIES_CODE | INDICATOR|
|-------------|----------|
|GGXONLB.A | Primary net lending (+) / net borrowing (-), General government, Domestic currency|
|GGXWDN_NGDP.A | Net debt, General government, Percent of GDP|
|LUR.A | Unemployment rate|
|GGR_NGDP.A | Revenue, General government, Percent of GDP|
|PPPEX.A | Rate, Domestic currency per international dollar in PPP terms, ICP benchmarks 2017-2021|
|GGSB_NPGDP.A | Structural balance, General government, Percent|
|GGXCNL_NGDP.A | Net lending (+) / net borrowing (-), General government, Percent of GDP|
|NGAP_NPGDP.A | Output gap, Percent of potential GDP|
|LE.A | Employed persons, Persons for countries / Index for country groups|
|GGXONLB_NGDP.A | Primary net lending (+) / net borrowing (-), General government, Percent of GDP|
|GGXWDG_NGDP.A | Gross debt, General government, Percent of GDP|
|GGX_NGDP.A | Expenditure, General government, Percent of GDP|
|TX_RPCH.A | Exports of goods and services, Volume, Free on board (FOB), Percent change|
|BCA_NGDPD.A | Current account balance (credit less debit), Percent of GDP|
|NID_NGDP.A | Gross capital formation, Percent of GDP|
|LP.A | Population, Persons for countries / Index for country groups|
|NGSD_NGDP.A | Gross national savings, Percent of GDP|
|PPPSH.A | Gross domestic product (GDP), Purchasing power parity (PPP) international dollar, percent of world GDP, Percent, ICP benchmarks 2017-2021|
|NGDP_RPCH.A | Gross domestic product (GDP), Constant prices, Percent change|
|TMG_RPCH.A | Imports of goods, Volume, Cost insurance freight (CIF), Percent change|
|TXG_RPCH.A | Exports of goods, Volume, Free on board (FOB), Percent change|
|BCA.A | Current account balance (credit less debit), US dollar|
|TM_RPCH.A | Imports of goods and services, Volume, Cost insurance freight (CIF), Percent change|
|NGDPPC.A | Gross domestic product (GDP), Current prices, Per capita, Domestic currency|
|NGDPRPC.A | Gross domestic product (GDP), Constant prices, Per capita, Domestic currency|
|PCPIE.A | All Items, Consumer price index (CPI), End-of-period (EoP)|
|PPPPC.A | Gross domestic product (GDP), Per capita, purchasing power parity (PPP) international dollar, ICP benchmarks 2017-2021|
|PCPIEPCH.A | All Items, Consumer price index (CPI), End-of-period (EoP), percent change|
|PCPI.A | All Items, Consumer price index (CPI), Period average|
|NGDP_D.A | Gross domestic product (GDP), Price deflator, Index|
|GGXWDG.A | Gross debt, General government, Domestic currency|
|GGSB.A | Structural balance, General government, Domestic currency|
|GGR.A | Revenue, General government, Domestic currency|
|NGDP_FY.A | Gross domestic product (GDP), Current prices, Fiscal year, Domestic currency|
|NGDPDPC.A | Gross domestic product (GDP), Current prices, Per capita, US dollar|
|NGDPD.A | Gross domestic product (GDP), Current prices, US dollar|
|PCPIPCH.A | All Items, Consumer price index (CPI), Period average, percent change|
|NGDP.A | Gross domestic product (GDP), Current prices, Domestic currency|
|GGX.A | Expenditure, General government, Domestic currency|
|NGDP_R.A | Gross domestic product (GDP), Constant prices, Domestic currency|
|PPPGDP.A | Gross domestic product (GDP), Current prices, Purchasing power parity (PPP) international dollar, ICP benchmarks 2017-2021|
|NGDPRPPPPC.A | Gross domestic product (GDP), Constant prices, Per capita, purchasing power parity (PPP) international dollar, ICP benchmark 2021|
|GGXCNL.A | Net lending (+) / net borrowing (-), General government, Domestic currency|
|GGXWDN.A | Net debt, General government, Domestic currency|

In [6]:
# import libraries
import sdmx

 

# retrieve data

IMF_DATA = sdmx.Client('IMF_DATA')

data_msg = IMF_DATA.data('WEO', key='BRA.LUR.A', params={'startPeriod': 2018})

 

# convert to pandas

cpi_df = sdmx.to_pandas(data_msg)

df = pd.DataFrame(cpi_df).reset_index()
df.head()

xml.Reader got no structure=… argument for StructureSpecificData


,INDICATOR,COUNTRY,FREQUENCY,LATEST_ACTUAL_ANNUAL_DATA,OVERLAP,SCALE,METHODOLOGY_NOTES,TIME_PERIOD,value
0,LUR,BRA,A,2024,OL,0,Source: National Statistics Office. Annual his...,2018,12.375
1,LUR,BRA,A,2024,OL,0,Source: National Statistics Office. Annual his...,2019,11.975
2,LUR,BRA,A,2024,OL,0,Source: National Statistics Office. Annual his...,2020,13.775
3,LUR,BRA,A,2024,OL,0,Source: National Statistics Office. Annual his...,2021,13.200
4,LUR,BRA,A,2024,OL,0,Source: National Statistics Office. Annual his...,2022,9.250




Series available in the national summary data table [NSDP](https://data.imf.org/en/datasets/IMF.STA:NSDP)
|SERIES_CODE | INDICATOR |
|------------|-----------|
|SNA.NGDP_Q_SA.XDC.Q | Gross domestic product, Constant prices, Seasonally adjusted (SA) |
|SNA.NGDP_V_NSA.XDC.Q | Gross domestic product, Current prices, Not seasonally adjusted (NSA) |
|LS.LE_PE.NUM.M | Employed persons |
|LS.LUR.PCH.M | Unemployment rate |
|LS.LEE.XDC.M | Monthly earnings / compensation |
|LS.LU_PE.NUM.M | Unemployed persons |
|LS.LLF_PE.NUM.M | Labor force (persons) |
|IPI.AIP_SA.IX.M | Industrial production index, Seasonally adjusted (SA) |
|PPI.PPPI.IX.M | Producer Price Index, All Commodities |
|CPI.PCPI.IX.M | Consumer Price,  All items |

In [ ]:
# import libraries
import sdmx

 

# retrieve data

IMF_DATA = sdmx.Client('IMF_DATA')

data_msg = IMF_DATA.data('NSDP', key='BRA.CPI.PCPI.IX.M', params={'startPeriod': 2018})

 

# convert to pandas

cpi_df = sdmx.to_pandas(data_msg)

df = pd.DataFrame(cpi_df).reset_index()
df

xml.Reader got no structure=… argument for StructureSpecificData


,INDICATOR,TYPE_OF_TRANSFORMATION,COUNTRY,NSDP_CAT,FREQUENCY,SCALE,TIME_PERIOD,value
0,PCPI,IX,BRA,CPI,M,0,2018-M01,4930.72
1,PCPI,IX,BRA,CPI,M,0,2018-M02,4946.50
2,PCPI,IX,BRA,CPI,M,0,2018-M03,4950.95
3,PCPI,IX,BRA,CPI,M,0,2018-M04,4961.84
4,PCPI,IX,BRA,CPI,M,0,2018-M05,4981.69
...,...,...,...,...,...,...,...,...
86,PCPI,IX,BRA,CPI,M,0,2025-M03,7245.38
87,PCPI,IX,BRA,CPI,M,0,2025-M04,7276.54
88,PCPI,IX,BRA,CPI,M,0,2025-M05,7295.46
89,PCPI,IX,BRA,CPI,M,0,2025-M06,7312.97
